In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt

from statsmodels.tools.eval_measures import rmse
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout  
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [32]:
train = pd.read_csv('datasets/podzielone/ml/wszystkie/train_ml.csv', index_col=0)
val1 = pd.read_csv('datasets/podzielone/ml/wszystkie/valid1_ml.csv', index_col=0)
val2 = pd.read_csv('datasets/podzielone/ml/wszystkie/valid2_ml.csv', index_col=0)
val3 = pd.read_csv('datasets/podzielone/ml/wszystkie/valid3_ml.csv', index_col=0)
test = pd.read_csv('datasets/podzielone/ml/wszystkie/test_ml.csv', index_col=0)




train.index = pd.to_datetime(train.index)
val1.index = pd.to_datetime(val1.index)
val2.index = pd.to_datetime(val2.index)
val3.index = pd.to_datetime(val3.index)
test.index = pd.to_datetime(test.index)


whole = pd.concat([train, val1, val2, val3, test]).asfreq('B', method='bfill').fillna(method='bfill')
val = pd.concat([val1, val2, val3]).asfreq('B', method='bfill').fillna(method='bfill')


train.columns=train.columns.str.replace('(','')
train.columns=train.columns.str.replace(']','')

val1.columns=val1.columns.str.replace('(','')
val1.columns=val1.columns.str.replace(']','')
val2.columns=val2.columns.str.replace('(','')
val2.columns=val2.columns.str.replace(']','')
val3.columns=val3.columns.str.replace('(','')
val3.columns=val3.columns.str.replace(']','')

val.columns=val.columns.str.replace('(','')
val.columns=val.columns.str.replace(']','')

train["day_of_week"] = train.index.dayofweek
train["day_of_year"] = train.index.dayofweek
train["week"] = train.index.week
train["quarter"] = train.index.quarter
train["return_ratio"] = train.y_return_ratio.shift()

val1["day_of_week"] = val1.index.dayofweek
val1["day_of_year"] = val1.index.dayofweek
val1["week"] = val1.index.week
val1["quarter"] = val1.index.quarter
val1["return_ratio"] = val1.y_return_ratio.shift()

val2["day_of_week"] = val2.index.dayofweek
val2["day_of_year"] = val2.index.dayofweek
val2["week"] = val2.index.week
val2["quarter"] = val2.index.quarter
val2["return_ratio"] = val2.y_return_ratio.shift()

val3["day_of_week"] = val3.index.dayofweek
val3["day_of_year"] = val3.index.dayofweek
val3["week"] = val3.index.week
val3["quarter"] = val3.index.quarter
val3["return_ratio"] = val3.y_return_ratio.shift()

test["day_of_week"] = test.index.dayofweek
test["day_of_year"] = test.index.dayofweek
test["week"] = test.index.week
test["quarter"] = test.index.quarter
test["return_ratio"] = test.y_return_ratio.shift()



train1 = train.asfreq('B', method='bfill').fillna(method='bfill')
train2 = pd.concat([train, val1]).asfreq('B', method='bfill').fillna(method='bfill')
train3 = pd.concat([train, val1, val2]).asfreq('B', method='bfill').fillna(method='bfill')

val1 = val1.asfreq('B', method='bfill').fillna(method='bfill')
val2 = val2.asfreq('B', method='bfill').fillna(method='bfill')
val3 = val3.asfreq('B', method='bfill').fillna(method='bfill')
test = test.asfreq('B', method='bfill').fillna(method='bfill')



display(train.head(3))
display(val1.head(3))
display(val2.head(3))
display(val3.head(3))
display(test.head(3))
display(whole.head(3))

,y_return_ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,...,ATAN_diff_1,CEIL_diff_1,FLOOR_diff_1,LN_diff_1,LOG10_diff_1,SQRT_diff_1,day_of_week,day_of_year,week,quarter
2012-07-02,-0.026773,0.475985,1.53,148.118434,12.781515,62.987843,-1.546392,59.313104,31.365741,2.350220,...,0.003153,0.0,0.0,0.042874,0.018620,0.078845,0,0,27,3
2012-07-03,0.026022,0.481986,1.05,103.024634,12.824644,57.664648,-22.613065,52.602204,23.907183,2.182347,...,-0.001980,0.0,0.0,-0.027138,-0.011786,-0.050102,1,1,27,3
2012-07-05,-0.010145,0.475416,0.56,110.859729,12.887570,61.020501,-5.025126,57.027455,23.907183,2.026465,...,0.001876,0.0,0.0,0.025689,0.011157,0.047411,3,3,27,3


,y_return_ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,...,ATAN_diff_1,CEIL_diff_1,FLOOR_diff_1,LN_diff_1,LOG10_diff_1,SQRT_diff_1,day_of_week,day_of_year,week,quarter
2018-01-02,0.030233,5.125354,7.029999,27.231458,196.912948,45.133485,-42.095010,55.894280,23.134279,13.144963,...,-0.000102,-4.0,-4.0,-0.019955,-0.008666,-0.139483,1,1,1,1
2018-01-03,0.065814,5.187829,7.790008,77.718696,197.070178,53.249819,-4.033801,55.819341,19.229973,13.246030,...,0.000152,6.0,6.0,0.029785,0.012935,0.208708,2,2,1,1
2018-01-04,0.005271,5.842269,14.570007,279.927608,198.063715,65.556102,-4.226792,67.077304,18.683056,26.499882,...,0.000310,13.0,13.0,0.063739,0.027681,0.457215,3,3,1,1


,y_return_ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,...,ATAN_diff_1,CEIL_diff_1,FLOOR_diff_1,LN_diff_1,LOG10_diff_1,SQRT_diff_1,day_of_week,day_of_year,week,quarter
2018-03-01,-0.040455,8.425720,9.369995,37.127358,234.729083,54.963241,-28.932093,70.942953,2.299899,24.419310,...,-0.000068,-5.0,-4.0,-0.016638,-0.007226,-0.129951,3,3,9,1
2018-03-02,0.018647,8.964597,-9.209991,-67.945451,234.566562,47.416343,-57.358478,71.264678,24.198760,22.675074,...,-0.000174,-9.0,-10.0,-0.041296,-0.017934,-0.317911,4,4,9,1
2018-03-05,-0.003763,9.392126,-9.960007,-108.267191,234.693880,50.644262,-51.228455,60.063802,31.886239,21.055426,...,0.000079,4.0,4.0,0.018475,0.008024,0.141418,0,0,10,1


,y_return_ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,...,ATAN_diff_1,CEIL_diff_1,FLOOR_diff_1,LN_diff_1,LOG10_diff_1,SQRT_diff_1,day_of_week,day_of_year,week,quarter
2018-05-01,0.009960,8.606211,-6.590011,-36.196616,229.129258,46.714722,-49.568247,52.298305,16.163336,22.665165,...,-0.000028,-2.0,-2.0,-0.006338,-0.002753,-0.047602,1,1,18,2
2018-05-02,-0.003654,8.352196,-10.399994,-41.609464,229.000918,48.681141,-41.830750,46.410723,19.251258,21.046224,...,0.000044,3.0,3.0,0.009911,0.004304,0.074498,2,2,18,2
2018-05-03,0.029517,8.009182,-10.059997,-16.440286,228.827311,47.974668,-44.697766,43.162233,15.551078,21.092926,...,-0.000016,-1.0,-1.0,-0.003661,-0.001590,-0.027561,3,3,18,2


,y_return_ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,...,ATAN_diff_1,CEIL_diff_1,FLOOR_diff_1,LN_diff_1,LOG10_diff_1,SQRT_diff_1,day_of_week,day_of_year,week,quarter
2018-07-02,0.022541,6.899792,-28.360016,-97.965938,250.580128,36.289331,-94.472074,39.345807,40.019379,13.617902,...,-0.000069,-4.0,-4.0,-0.016578,-0.007200,-0.128109,0,0,27,3
2018-07-03,-0.022292,7.059093,-22.849991,-81.994860,250.042056,42.388575,-75.013896,39.426792,44.725629,12.645195,...,0.000093,6.0,6.0,0.022291,0.009681,0.172505,1,1,27,3
2018-07-04,0.024869,7.082015,-23.330017,-71.397983,249.190310,38.386549,-90.038956,34.350825,39.822012,13.231972,...,-0.000094,-6.0,-6.0,-0.022544,-0.009791,-0.174455,3,3,27,3


,y_return_ratio,ATR,MOM,CCI,EMA,RSI,Will_R,MFI,DX,PLUS_DM,...,MAXINDEX_diff_1,MIN_diff_1,MININDEX_diff_1_shift_19,MULT_diff_1,ATAN_diff_1,CEIL_diff_1,FLOOR_diff_1,LN_diff_1,LOG10_diff_1,SQRT_diff_1
2012-07-02,-0.026773,0.475985,1.53,148.118434,12.781515,62.987843,-1.546392,59.313104,31.365741,2.350220,...,7.0,0.0,1.0,15.5678,0.003153,0.0,0.0,0.042874,0.018620,0.078845
2012-07-03,0.026022,0.481986,1.05,103.024634,12.824644,57.664648,-22.613065,52.602204,23.907183,2.182347,...,0.0,0.0,0.0,-2.1030,-0.001980,0.0,0.0,-0.027138,-0.011786,-0.050102
2012-07-04,-0.010145,0.475416,0.56,110.859729,12.887570,61.020501,-5.025126,57.027455,23.907183,2.026465,...,0.0,0.0,0.0,0.4492,0.001876,0.0,0.0,0.025689,0.011157,0.047411


In [33]:
S = [(train1, val1), (train2, val2), (train3, val3)]

In [34]:
variables = train.columns.tolist()
variables.remove("y_return_ratio")
features = train[variables].columns
target = 'y_return_ratio'

# Korelacje Spearmana

In [35]:
corr_matrix = whole.drop(columns=['y_return_ratio']).corr("spearman").abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

var1 = [col for col in upper.columns if upper[col].max() < 0.9]
var2 = [col for col in upper.columns if upper[col].max() < 0.7]
var3 = [col for col in upper.columns if upper[col].max() < 0.5]
var4 = [col for col in upper.columns if upper[col].max() < 0.4]
var5 = [col for col in upper.columns if upper[col].max() < 0.3]

print(len(var1))
print(len(var2))
print(len(var3))
print(len(var4))
print(len(var5))

188
126
98
77
56


In [36]:
whole.columns.tolist()

['y_return_ratio',
 'ATR',
 'MOM',
 'CCI',
 'EMA',
 'RSI',
 'Will_R',
 'MFI',
 'DX',
 'PLUS_DM',
 'PPO',
 'AROONOSC',
 'BOP',
 'MINUS_DM',
 'ULTOSC',
 'AD',
 'OBV',
 'DEMA',
 'HT_TRENDLINE',
 'KAMA',
 'MIDPOINT',
 'MIDPRICE',
 'SAR',
 'SAREXT',
 'T3',
 'TEMA',
 'TRIMA',
 'WMA',
 'ADX',
 'ADXR',
 'APO',
 'CMO',
 'MINUS_DI',
 'PLUS_DI',
 'ROC',
 'ROCP',
 'ROCR',
 'ROCR100',
 'ADOSC',
 'NATR',
 'TRANGE',
 'AVGPRICE',
 'MEDPRICE',
 'TYPPRICE',
 'WCLPRICE',
 'HT_DCPERIOD',
 'HT_DCPHASE',
 'HT_TRENDMODE',
 'CDL2CROWS',
 'CDL3BLACKCROWS',
 'CDL3INSIDE',
 'CDL3LINESTRIKE',
 'CDL3OUTSIDE',
 'CDL3STARSINSOUTH',
 'CDL3WHITESOLDIERS',
 'CDLABANDONEDBABY',
 'CDLADVANCEBLOCK',
 'CDLBELTHOLD',
 'CDLBREAKAWAY',
 'CDLCLOSINGMARUBOZU',
 'CDLCONCEALBABYSWALL',
 'CDLCOUNTERATTACK',
 'CDLDARKCLOUDCOVER',
 'CDLDOJI',
 'CDLDOJISTAR',
 'CDLDRAGONFLYDOJI',
 'CDLENGULFING',
 'CDLEVENINGDOJISTAR',
 'CDLEVENINGSTAR',
 'CDLGAPSIDESIDEWHITE',
 'CDLGRAVESTONEDOJI',
 'CDLHAMMER',
 'CDLHANGINGMAN',
 'CDLHARAMI',
 'CDL

# ExtraTreesRegression

In [37]:
X_train1 = train1[var1]
y_train1 = train1[target]
X_test1 = val[var1]
y_test1 = val[target]

KeyError: "['NVIDIA_news_inni_o_nich_pisza_(1, 3]', 'NVIDIA_news_inni_o_nich_pisza_(0, 1]', 'ogloszenie_nowej_karty_(0, 1]', 'NVIDIA_news_publikacja_(0, 1]', 'NVIDIA_news_inni_o_nich_pisza_(3, 6]', 'NVIDIA_news_publikacja_(3, 6]', 'ogloszenie_nowej_karty_(3, 20]', 'NVIDIA_news_publikacja_(1, 3]', 'NVIDIA_news_publikacja_(6, 100]', 'ogloszenie_nowej_karty_(1, 3]'] not in index"

In [ ]:
model = ExtraTreesRegressor()
model.fit(X_train1, y_train1)
feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances

# LSTM

In [9]:
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
scalerX = MinMaxScaler(feature_range=(0, 1))
scalerY = MinMaxScaler(feature_range=(0, 1))
end = int(len(var1)/10)
for i in range(1, 2):
#     variables_in = feature_importances.index[:20]
    trainX = train1[['AROONOSC',
   'CDLENGULFING',
   'CDLMATCHINGLOW',
   'CDLSHORTLINE',
   'MININDEX']]
    trainY = train1['y_return_ratio']
    
    valX = val[['AROONOSC',
   'CDLENGULFING',
   'CDLMATCHINGLOW',
   'CDLSHORTLINE',
   'MININDEX']]
    valY = val['y_return_ratio']
    
    features_set = scalerX.fit_transform(trainX)
    features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
    labels = scalerY.fit_transform(np.array(trainY).reshape(-1, 1))
    
    test_features_set = scalerX.transform(valX)
    test_features_set = np.reshape(test_features_set, (test_features_set.shape[0], test_features_set.shape[1], 1))
    
    model = Sequential()
    
    model.add(LSTM(units=test_features_set.shape[1], input_shape=(features_set.shape[1], 1)))
    
    model.add(Dense(units = 1))
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    opt = SGD(lr=0.01)
    
    model.compile(optimizer = opt, loss = 'mse')
    model.fit(features_set, labels, epochs = 100, batch_size = 32, validation_data=(test_features_set, valY), callbacks=[es], verbose=1)
    
    predictions = model.predict(test_features_set)
    predictions = scalerY.inverse_transform(predictions)
    
    plt.figure(figsize=(10,6))
    pd.Series([x[0] for x in predictions], index=y_test1.index).plot(label='pred')
    y_test1.plot(label="real")
    plt.legend()
    plt.show()
    print(rmse([x[0] for x in predictions], y_test1))
#     plt.plot(predictions)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1435 samples, validate on 129 samples
Epoch 1/100
1435/1435 [==============================] - 1s 440us/step - loss: 0.0180 - val_loss: 0.0403
Epoch 2/100
1435/1435 [==============================] - 0s 96us/step - loss: 0.0035 - val_loss: 0.0522
Epoch 3/100
1435/1435 [==============================] - 0s 96us/step - loss: 0.0032 - val_loss: 0.0542
Epoch 4/100
1435/1435 [==============================] - 0s 111us/step - loss: 0.0032 - val_loss: 0.0547
Epoch 5/100
1435/1435 [==============================] - 0s 103us/step - loss: 0.0032 - val_loss: 0.0548
Epoch 6/100
1435/1435 [==============================] - 0s 104us/step - loss: 0.0032 - val_loss: 0.0544
Epoch 7/100
1435/1435 [==============================] - 0s 85us/step - loss: 0.0032 - val_loss: 0.0539
Epoch 8/100
1435/1435 [==============================] - 0s 88us/step - loss: 0.0032 - val_loss: 0.05

NameError: name 'y_test1' is not defined

<Figure size 720x432 with 0 Axes>

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances